## Esta es una versión modificada del ejemplo descargado de github, para adaptarlo a mi necesidad (electricity/rto/fuel-type-data)

## Get desired data endpoint

https://www.eia.gov/opendata/browser/electricity/rto/fuel-type-data

## Initialize route in Python

In [ ]:
from eia import EIAClient

client = EIAClient() 
route = client.get_data_endpoint('electricity/rto/fuel-type-data') # línea modificada respecto al modelo de ejemplo descargado

2025-08-04 22:42:58,610 - INFO - EIAClient initialized.
2025-08-04 22:42:58,612 - INFO - Directly accessing data endpoint metadata for: electricity/rto/fuel-type-data
2025-08-04 22:42:58,613 - INFO - Fetching metadata for route: electricity/rto/fuel-type-data


https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000

## List filtering options

### Facets: column values

In [ ]:
route.facets


FacetContainer(facets=[respondent, fueltype])

In [ ]:
facet_options = route.facets.respondent.get_values() # línea modificada respecto al modelo de ejemplo descargado
facet_options

# Acá voy a ver que CISO es California (CISO lo uso en las celdas siguientes)


2025-08-04 22:45:37,435 - INFO - Fetching facet values for facet 'respondent' in route: electricity/rto/fuel-type-data


[FacetValue(id='NY', name='New York', description=None),
 FacetValue(id='NWMT', name='NorthWestern Corporation', description=None),
 FacetValue(id='SCEG', name='Dominion Energy South Carolina, Inc.', description=None),
 FacetValue(id='TEPC', name='Tucson Electric Power', description=None),
 FacetValue(id='MIDA', name='Mid-Atlantic', description=None),
 FacetValue(id='FMPP', name='Florida Municipal Power Pool', description=None),
 FacetValue(id='NYIS', name='New York Independent System Operator', description=None),
 FacetValue(id='LDWP', name='Los Angeles Department of Water and Power', description=None),
 FacetValue(id='CAL', name='California', description=None),
 FacetValue(id='CISO', name='California Independent System Operator', description=None),
 FacetValue(id='PSEI', name='Puget Sound Energy, Inc.', description=None),
 FacetValue(id='CAR', name='Carolinas', description=None),
 FacetValue(id='GVL', name='Gainesville Regional Utilities', description=None),
 FacetValue(id='NEVP', na

### Frequencies: datetime intervals

In [4]:
route.frequencies

[FrequencyInfo(id='hourly', description='One data point for each hour in UTC time.', query='H', format='YYYY-MM-DD"T"HH24'),
 FrequencyInfo(id='local-hourly', description='One data point for each hour in local time.', query='LH', format='YYYY-MM-DD"T"HH24TZH')]

## Query data

In [5]:
# Hago la consulta para CISO, que es California (ver celdas más arriba)

df_ciso = route.get(
    data_columns=['value'],
    facets={'respondent': 'CISO'},      # línea modificada respecto al modelo de ejemplo descargado
    frequency='hourly',
    sort=[
        {'column': 'period','direction': 'desc'},
        {'column': 'fueltype', 'direction': 'asc'}     # línea modificada respecto al modelo de ejemplo descargado
    ]
)

df_ciso


2025-08-04 22:49:46,833 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 0, Length: All (Pagination)
2025-08-04 22:49:46,834 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-08-04 22:49:48,744 - INFO - Received 5000 rows.
2025-08-04 22:49:48,745 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 5000, Length: All (Pagination)
2025-08-04 22:49:48,745 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-08-04 22:49:50,487 - INFO - Received 5000 rows.
2025-08-04 22:49:50,488 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 10000, Length: All (Pagination)
2025-08-04 22:49:50,489 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-08-04 22:49:51,980 - INFO - Received 5000 rows.
2025-08-04 22:49:51,981 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 15000, Length: All (Pagination)
2025-08-04 22:49:51,982 - INFO - Fetchin

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2025-08-04 06:00:00+00:00,CISO,California Independent System Operator,COL,Coal,0.0,megawatthours
1,2025-08-04 06:00:00+00:00,CISO,California Independent System Operator,NG,Natural Gas,7925.0,megawatthours
...,...,...,...,...,...,...,...
446164,2019-01-01 00:00:00+00:00,CISO,California Independent System Operator,WAT,Hydro,2403.0,megawatthours
446165,2019-01-01 00:00:00+00:00,CISO,California Independent System Operator,WND,Wind,2497.0,megawatthours


Esta librería gestiona automáticamente los límites de 5.000 registros, haciendo múltiples descargas hasta llegar al total del dataset.

## Export data

In [6]:
df_ciso.period

0        2025-08-04 06:00:00+00:00
1        2025-08-04 06:00:00+00:00
                    ...           
446164   2019-01-01 00:00:00+00:00
446165   2019-01-01 00:00:00+00:00
Name: period, Length: 446166, dtype: datetime64[ns, UTC]

In [7]:
df_ciso.period.dt.tz_convert('America/Los_Angeles')

0        2025-08-03 23:00:00-07:00
1        2025-08-03 23:00:00-07:00
                    ...           
446164   2018-12-31 16:00:00-08:00
446165   2018-12-31 16:00:00-08:00
Name: period, Length: 446166, dtype: datetime64[ns, America/Los_Angeles]

In [8]:
df_ciso.period = df_ciso.period.dt.tz_convert('America/Los_Angeles')

In [9]:
df_ciso.to_csv('../../../data/EIA/energy_by_fueltype_ciso.csv', index=False)
